## Global Sensitivity and Uncertainty Analysis

The GSUA approach is based on the work of  A. Carmona-Cabrero and R. Muñoz-Carpena, University of Florida. The paper can be found  in JASS:  [(Carmona-Cabrero et al., 2024)](https://doi.org/10.18564/jasss.5174) 



In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from SALib.sample import sobol
from ring_plot_funct import ring_plot
import json
from pathlib import Path

# Folder containing config files
CONFIG_DIR = Path("/blue/carpena/haasehelen/ifwaste/input/gsua_based_configuration")


In [6]:
def extract_parameters(d, path=""):
    result = {}
    for key, value in d.items():
        current_path = f"{path}:{key}" if path else key

        if isinstance(value, dict):
            if "value" in value and "layer" in value:
                result[current_path] = {
                    "value": value.get("value", "NAN"),
                    "distribution":  value.get("distribution", "NAN"),
                    "bounds":  value.get("bounds", "NAN"),
                    "layer": value["layer"],
                    "dtype": value.get("dtype", "NAN"),
                    "length": value.get("length", "NAN"),
                    "options":value.get("options", "NAN"),
                    "decimals": value.get("decimals", "NAN")
                }
            elif "distribution" in value:
                result[current_path] = {
                    "value":  value.get("value", "NAN"),
                    "distribution": value["distribution"],
                    "bounds":  value.get("bounds", "NAN"),
                    "layer": value.get("layer", value["distribution"]),
                    "dtype": value.get("dtype", "NAN"),
                    "length": value.get("length", "NAN"),
                    "options":value.get("options", "NAN"),
                    "decimals": value.get("decimals", "NAN")
                }
            else:
                nested_result = extract_parameters(value, current_path)
                result.update(nested_result)
    return result

## 1. First read all parameterization values from the json files in the CONFIG_DIR
Generate 4 dfs: 
- constants: all parameters that are constants
- parameter: all parameters that shall be sampled using sobol split into:
  - simple_distributions: all parameters that are either triangular or uniform
  - other_distribtuions: parameters, that require extra attention, as they have to be converted to a "simple" distribution


In [7]:
all_json_files = list(CONFIG_DIR.glob("*.json"))
df_list = []
for json_file in all_json_files:
    with open(json_file) as f:
        config = json.load(f)
    flat_params = extract_parameters(config)
    df = pd.DataFrame.from_dict(flat_params, orient="index")
    df.index.name = "name"
    df.reset_index(inplace=True)
    df.replace("", "NAN", inplace=True)
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)

constants = df[df["value"] != "NAN"]
parameter = df[df["value"] == "NAN"]
mask = (parameter["distribution"] == "triang") | (parameter["distribution"] == "unif")
simple_distributions = parameter[mask ]
other_distributions = parameter[~(mask)]



In [8]:
#sim = df[df["layer"] == "simulation"]
#sim = sim.drop(columns=["bounds", "distribution", "layer", "length", "options"])
#sim.to_latex(index=False)

In [9]:
#nh = df[df["layer"] == "neighborhood"]
#nh = nh.drop(columns=["value", "layer", "length", "dtype", "decimals"])
#nh.to_latex(index=False)

In [10]:
#hh = df[df["layer"] == "household"]
#hh = hh.drop(columns=["value", "layer", "length", "options"])
#hh.to_latex()

In [11]:
simple_distributions.head(10)

,name,value,distribution,bounds,layer,dtype,length,options,decimals
9,Premium_retailer:Sales:high_stock_interval_1,NAN,unif,"[1.3, 1.5]",neighborhood,float,NAN,NAN,2
13,Premium_retailer:Sales:seasonal_likelihood,NAN,unif,"[0, 0.05]",neighborhood,float,NAN,NAN,2
25,Convenience_store:Sales:high_stock_interval_1,NAN,unif,"[1, 1.5]",neighborhood,float,NAN,NAN,2
29,Convenience_store:Sales:seasonal_likelihood,NAN,unif,"[0, 0.25]",neighborhood,float,NAN,NAN,2
41,Neighborhood:Grid:travel_time_per_cell,NAN,triang,"[3, 7, 0.5]",neighborhood,float,NAN,NAN,1
45,Neighborhood:BasketCurator:increment_likelihood,NAN,unif,"[0.01, 1]",neighborhood,float,NAN,NAN,2
46,Neighborhood:BasketCurator:max_items_quickshop,NAN,unif,"[2, 8]",neighborhood,int,NAN,NAN,NAN
91,Household:hh_amount_children,NAN,unif,"[0, 6]",household,int,NAN,NAN,NAN
92,Household:hh_amount_adults,NAN,unif,"[1, 2]",household,int,NAN,NAN,NAN
93,Household:hh_level_of_concern,NAN,unif,"[0, 1]",household,float,NAN,NAN,2


## 2. Convert all parameters from "other_distribution" into "simple_distribtuions"

In [12]:
#convert other distributions to sobol-sampleable values
for _,row in other_distributions.iterrows(): 
    if row["distribution"] == "subgroup": 
        new_row = pd.DataFrame({"name":row["name"], "distribution":"unif", "bounds":[[0,1]], "dtype":"float",
                            "decimals":2, "layer": row["layer"], "value":"NAN", "options":"NAN"})
        simple_distributions = pd.concat([simple_distributions, new_row], ignore_index=True)
        
    if row["distribution"] == "vector": 
        for i in range(row["length"]): 
            new_row = pd.DataFrame({"name":row["name"]+"_"+str(i), "distribution":"unif", "bounds":[row["bounds"]], "dtype":row["dtype"],
                            "decimals":2, "layer": row["layer"], "value":"NAN", "options":"NAN"})
            simple_distributions = pd.concat([simple_distributions, new_row], ignore_index=True)
            
    if row["distribution"] == "sales": 
        new_row = pd.DataFrame({"name":row["name"], "distribution":"unif", "bounds":[[0,1]], "dtype":"float",
                            "decimals":2, "layer": row["layer"], "value":"NAN", "options":"NAN"})
        #sales werden dann von vorne nach hinten ausgewählt - nicht noch randomisiert - zack 
        simple_distributions = pd.concat([simple_distributions, new_row], ignore_index=True)        
    

In [13]:
#sample other distributions by hand: subgroup
#import random

#def get_value_for_subgroup(bounds):
## Step 1: Extract ranges and probabilities
#    ranges = []
#    probs = []
#    for key, prob in bounds.items():
#        start, end = map(int, key.split('-'))
#        ranges.append((start, end))
#        probs.append(prob)
#    chosen_range = random.choices(ranges, weights=probs, k=1)[0]
#    sampled_value = random.randint(chosen_range[0], chosen_range[1])

#for row in other_distributions.rows: 
#    if row["distribution"] == "subgroup":
#        row["value"] = get_value_for_subgroup(row["bounds"])
#    elif row["distribution"] == "vector": 
#        if row["name"].contains["store_amounts"]: 
            

In [14]:
'''import random

def get_value_for_subgroup(bounds):
    ranges = []
    probs = []

    for key, prob in bounds.items():
        if '-' in key:
            start, end = map(int, key.split('-'))
        else:
            start = end = int(key)  # Single value case
        ranges.append((start, end))
        probs.append(prob)

    # Choose a range/value based on probability
    chosen_range = random.choices(ranges, weights=probs, k=1)[0]
    sampled_value = random.randint(chosen_range[0], chosen_range[1])
    
    return sampled_value
print(get_value_for_subgroup({'14': 0.628, '30': 0.103, '7': 0.269}))'''

"import random\n\ndef get_value_for_subgroup(bounds):\n    ranges = []\n    probs = []\n\n    for key, prob in bounds.items():\n        if '-' in key:\n            start, end = map(int, key.split('-'))\n        else:\n            start = end = int(key)  # Single value case\n        ranges.append((start, end))\n        probs.append(prob)\n\n    # Choose a range/value based on probability\n    chosen_range = random.choices(ranges, weights=probs, k=1)[0]\n    sampled_value = random.randint(chosen_range[0], chosen_range[1])\n    \n    return sampled_value\nprint(get_value_for_subgroup({'14': 0.628, '30': 0.103, '7': 0.269}))"

## 3. Read all parameters (now simple and other) into a problem definition for sobol sampling
Then round all values according to the json files (dtype + decimals)

## 4. Prepare for HPC
- add sens_class line for HPC
- check if at least one store would be in the neighborhood, otherwise remove entire sample

## 5. Add replications of samples, write everything to the file
- save the values by level
- also save the parameter names in an extra file for later usage

## 6. Convert other_distributions back to original types for simulation

In [15]:
import csv


l = 4  # Sampling intensity
N = 10
SAVE_DIR = "/blue/carpena/haasehelen/ifwaste/input/gsua_based_configuration/samples/"


def convert_p_to_value_in_subgroup(p, bounds): 
    cumulative = 0.0
    for key, prob in bounds.items():
        if '-' in key:
            start, end = map(int, key.split('-'))
        else:
            start = end = int(key)
        cumulative += prob
        if p <= cumulative:
            n_values = end-start+1
            for i in range(n_values):
                cum_in_group = (cumulative-prob) + i * (prob/n_values)
                if p <= cum_in_group or start == end:
                    return start + i

problems = {}

for layer in simple_distributions["layer"].unique():
    print(layer)
    df = simple_distributions[simple_distributions["layer"] == layer]
    names = df["name"].to_list()
    distributions = df["distribution"].to_list()
    bounds = df["bounds"].to_list()
    problem = {'num_vars': len(names),
    'names': names,
    'bounds': bounds,
    'dists':distributions
    }
    param_values = pd.DataFrame(sobol.sample(problem, l, calc_second_order=False), columns=names)  # calc_second_order=True for second-order interactions. 
    
    #round values
    for param_name in param_values.columns: 
        row = df[df["name"] == param_name]
        if row["dtype"].values[0] == "int":
            param_values[param_name] = param_values[param_name].round().astype("int")
        elif row["dtype"].values[0] == "float": 
            param_values[param_name] = param_values[param_name].astype("float").round(int(row["decimals"].values[0]))
    
    #filter out unperformable sets (no stores in simulation)
    if layer == "neighborhood":
        print(len(param_values))
        param_values = param_values[~((param_values['Neighborhood:nh_store_amounts_0'] + param_values['Neighborhood:nh_store_amounts_1'] + param_values['Neighborhood:nh_store_amounts_2']) == 0)]
        print(len(param_values))
        
        
    #transform other distributions back
    for _,row in other_distributions.iterrows(): 
        if row["distribution"] == "subgroup" and row["name"] in param_values.columns: 
            bounds = row["bounds"]
            df = pd.DataFrame()
            df["probability"] = param_values[row["name"]]
            df["results"] = df["probability"].apply(lambda p: convert_p_to_value_in_subgroup(p, bounds))
            param_values[row["name"]] = df["results"]
            
        if row["distribution"] == "vector" and f"{row["name"]}_{0}" in param_values.columns:
            columns_to_drop = []
            col_names = [f"{row["name"]}_{i}" for i in range(row["length"])]
            # Create new column with vectors (lists) for each row
            param_values[row["name"]] = param_values[col_names].values.tolist()

            # Queue old columns for dropping
            columns_to_drop.extend(col_names)
            param_values.drop(columns=columns_to_drop, inplace=True)
            
        if row["distribution"] == "sales"  and row["name"] in param_values.columns: 
            percentage = param_values[row["name"]]
            df = pd.DataFrame()
            # Calculate how many sales to select
            df["number_of_sales"] = round(percentage * len(row["options"])).astype(int)
            df["number_of_sales"] = df["number_of_sales"].clip(lower=1)
            df["selected_sales"] = df["number_of_sales"].apply(lambda n: row["options"][:n])
            param_values[row["name"]] = df["selected_sales"]
    
    print("GSA sample matrix size: {}".format(param_values.shape))  
    #for HPC added "sens_class" line
    header = param_values.columns.to_list()
    #param_values['sens_class'] = 'sens_class'
    param_values.insert(0, 'sens_class', 'sens_class')
    sample_matrix_realizations = pd.DataFrame(np.repeat(param_values.to_numpy(), repeats=N, axis=0))
    
    print("GSA sample matrix size: {}".format(sample_matrix_realizations.shape))
    #write the sample matrix to csv to configure the model
    sample_matrix_realizations.to_csv( f"{SAVE_DIR}{layer}_ifwaste_sample.txt", header=False, index=False, sep='\t')
    with open(f"{SAVE_DIR}{layer}_ifwaste_sample_header.txt", "w", newline="") as f:
        writer = csv.writer(f)
        for item in header:
            writer.writerow([item])
    
    
constants["value"].to_csv( SAVE_DIR + 'simulation_ifwaste_sample.txt', header=False, index=False, sep='\t')
constants["name"].to_csv( SAVE_DIR + 'simulation_ifwaste_sample_header.txt', index=False, header=False)
        

neighborhood
92
89
GSA sample matrix size: (89, 19)
GSA sample matrix size: (890, 20)
household
GSA sample matrix size: (192, 28)
GSA sample matrix size: (1920, 29)


In [16]:
param_values

,sens_class,Household:hh_amount_children,Household:hh_amount_adults,Household:hh_level_of_concern,Household:hh_impulse_buy_likelihood,Household:hh_daily_budget,Household:hh_min_time_to_cook,Household:hh_time_per_store,Household:hh_price_sensitivity,Household:hh_brand_sensitivity,...,Child:child_plate_waste,Child:male_store_prepared_ratio,Child:female_store_prepared_ratio,Cooking:cook_max_scaler_cooking_amount,Cooking:cook_expiration_threshold,Household:hh_shopping_frequency,Household:hh_pay_day_interval,Household:hh_max_avail_time_per_day,Adult:adult_preference_vector,Child:child_preference_vector
0,sens_class,1,2,0.05,1.30,58.68,20,22,0.28,0.40,...,0.320,0.12,0.32,1.89,3.91,NaN,7,"[115, 111, 161, 68, 135, 89, 64]","[0.71, 0.63, 0.18, 0.02, 0.33, 0.56, 0.32]","[0.66, 0.05, 0.2, 0.9, 0.89, 0.5, 0.37]"
1,sens_class,3,2,0.05,1.30,58.68,20,22,0.28,0.40,...,0.320,0.12,0.32,1.89,3.91,NaN,7,"[115, 111, 161, 68, 135, 89, 64]","[0.71, 0.63, 0.18, 0.02, 0.33, 0.56, 0.32]","[0.66, 0.05, 0.2, 0.9, 0.89, 0.5, 0.37]"
2,sens_class,1,2,0.05,1.30,58.68,20,22,0.28,0.40,...,0.320,0.12,0.32,1.89,3.91,NaN,7,"[115, 111, 161, 68, 135, 89, 64]","[0.71, 0.63, 0.18, 0.02, 0.33, 0.56, 0.32]","[0.66, 0.05, 0.2, 0.9, 0.89, 0.5, 0.37]"
3,sens_class,1,2,0.43,1.30,58.68,20,22,0.28,0.40,...,0.320,0.12,0.32,1.89,3.91,NaN,7,"[115, 111, 161, 68, 135, 89, 64]","[0.71, 0.63, 0.18, 0.02, 0.33, 0.56, 0.32]","[0.66, 0.05, 0.2, 0.9, 0.89, 0.5, 0.37]"
4,sens_class,1,2,0.05,0.67,58.68,20,22,0.28,0.40,...,0.320,0.12,0.32,1.89,3.91,NaN,7,"[115, 111, 161, 68, 135, 89, 64]","[0.71, 0.63, 0.18, 0.02, 0.33, 0.56, 0.32]","[0.66, 0.05, 0.2, 0.9, 0.89, 0.5, 0.37]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,sens_class,2,1,0.55,0.18,78.54,23,39,0.16,0.23,...,0.445,0.22,0.11,1.32,2.63,3.0,14,"[30, 36, 120, 99, 149, 163, 55]","[0.31, 0.97, 0.5, 0.45, 0.73, 0.27, 0.9]","[0.28, 0.94, 0.33, 0.65, 0.32, 0.95, 0.73]"
188,sens_class,2,1,0.55,0.18,78.54,23,39,0.16,0.23,...,0.445,0.22,0.11,1.32,2.63,3.0,14,"[30, 36, 120, 99, 149, 163, 55]","[0.31, 0.97, 0.5, 0.45, 0.73, 0.27, 0.9]","[0.28, 0.94, 0.33, 0.21, 0.89, 0.95, 0.73]"
189,sens_class,2,1,0.55,0.18,78.54,23,39,0.16,0.23,...,0.445,0.22,0.11,1.32,2.63,3.0,14,"[30, 36, 120, 99, 149, 163, 55]","[0.31, 0.97, 0.5, 0.45, 0.73, 0.27, 0.9]","[0.28, 0.94, 0.33, 0.21, 0.32, 0.24, 0.73]"
190,sens_class,2,1,0.55,0.18,78.54,23,39,0.16,0.23,...,0.445,0.22,0.11,1.32,2.63,3.0,14,"[30, 36, 120, 99, 149, 163, 55]","[0.31, 0.97, 0.5, 0.45, 0.73, 0.27, 0.9]","[0.28, 0.94, 0.33, 0.21, 0.32, 0.95, 0.97]"
